# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,2.86,91,49,6.71,CL,1726367935
1,1,port-aux-francais,-49.3500,70.2167,0.47,64,91,14.96,TF,1726367937
2,2,bahia honda,22.9036,-83.1592,27.43,83,44,1.72,CU,1726367938
3,3,kinmparana,12.8402,-4.9199,23.89,89,37,1.28,ML,1726367939
4,4,grytviken,-54.2811,-36.5092,-2.10,77,21,4.88,GS,1726367940


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_map = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True,
    size = "Humidity",
    color = "blue",
    frame_height=600,
    frame_width=800, tiles='OSM', alpha=0.6, hover_cols=['City', 'Country', 'Humidity'])
# Display the map
# YOUR CODE HERE
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_cities = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_cities = ideal_weather_cities.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,antonio enes,-16.2325,39.9086,22.68,85,0,3.09,MZ,1726368024
79,79,al bawiti,28.3492,28.8659,23.58,67,0,2.36,EG,1726368025
80,80,newman,37.3138,-121.0208,22.81,42,0,3.41,US,1726368026
93,93,mersa matruh,31.3525,27.2453,24.11,78,0,2.06,EG,1726368042
164,164,uturoa,-16.7333,-151.4333,26.62,75,0,3.98,PF,1726368129


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_cities[["City", "Country", 'Lat', "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ''

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
78,antonio enes,MZ,-16.2325,39.9086,85,
79,al bawiti,EG,28.3492,28.8659,67,
80,newman,US,37.3138,-121.0208,42,
93,mersa matruh,EG,31.3525,27.2453,78,
164,uturoa,PF,-16.7333,-151.4333,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
antonio enes - nearest hotel: Hotel Quirimbas
al bawiti - nearest hotel: Old Oasis Hotel
newman - nearest hotel: No hotel found
mersa matruh - nearest hotel: صقور
uturoa - nearest hotel: Hawaiki Nui hotel
ashland - nearest hotel: Quality Inn
remire-montjoly - nearest hotel: Complexe Belova
errachidia - nearest hotel: No hotel found
tucumcari - nearest hotel: Pow Wow Inn
sona - nearest hotel: Al Bosco
al jawf - nearest hotel: No hotel found
griffith - nearest hotel: Quality Inn & Suites Hammond
nakambala - nearest hotel: No hotel found
platteville - nearest hotel: Holiday Inn Express & Suites Platteville
nova vicosa - nearest hotel: Pousada Pontal da Barra
khuzdar - nearest hotel: Al-Mustafa hotel
emerald - nearest hotel: Emerald Central Hotel
tallulah - nearest hotel: Days Inn
tsiombe - nearest hotel: No hotel found
rio de janeiro - nearest hotel: Ibis Rio porto atlantico


,City,Country,Lat,Lng,Humidity,Hotel Name
78,antonio enes,MZ,-16.2325,39.9086,85,Hotel Quirimbas
79,al bawiti,EG,28.3492,28.8659,67,Old Oasis Hotel
80,newman,US,37.3138,-121.0208,42,No hotel found
93,mersa matruh,EG,31.3525,27.2453,78,صقور
164,uturoa,PF,-16.7333,-151.4333,75,Hawaiki Nui hotel
190,ashland,US,40.8167,-82.2665,84,Quality Inn
257,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
303,errachidia,MA,31.9275,-4.4285,46,No hotel found
357,tucumcari,US,35.1717,-103.7250,31,Pow Wow Inn
365,sona,IT,45.4388,10.8255,23,Al Bosco


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
country_hotel_name_map = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True,
    size="Humidity",
    color="blue",
    frame_height=600,
    frame_width=800,
    tiles='OSM',
    alpha=0.6,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'])


# Display the map
# YOUR CODE HERE
country_hotel_name_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)